In [1]:
import numpy as np
import random

class AntColonyOptimization:
    def __init__(self, num_ants, num_iterations, alpha, beta, evaporation_rate, Q, graph):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha  # Pheromone influence
        self.beta = beta    # Heuristic influence (e.g., inverse of distance/cost)
        self.evaporation_rate = evaporation_rate
        self.Q = Q          # Pheromone deposit constant
        self.graph = graph  # Adjacency matrix or similar representation
        self.num_nodes = len(graph)
        self.pheromones = np.ones((self.num_nodes, self.num_nodes)) # Initialize pheromones

    def run(self, start_node, end_node):
        best_path = None
        best_path_cost = float('inf')

        for iteration in range(self.num_iterations):
            all_paths = []
            for ant in range(self.num_ants):
                path, cost = self._construct_path(start_node, end_node)
                all_paths.append((path, cost))

                if cost < best_path_cost:
                    best_path_cost = cost
                    best_path = path

            self._update_pheromones(all_paths)
            self._evaporate_pheromones()

        return best_path, best_path_cost

    def _construct_path(self, start, end):
        current_node = start
        path = [current_node]
        visited = {current_node}
        path_cost = 0

        while current_node != end:
            probabilities = self._calculate_probabilities(current_node, visited)
            if not probabilities: # No valid path
                return [], float('inf')

            next_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()), k=1)[0]
            path.append(next_node)
            path_cost += self.graph[current_node][next_node]
            visited.add(next_node)
            current_node = next_node
        return path, path_cost

    def _calculate_probabilities(self, current_node, visited):
        probabilities = {}
        total_heuristic_pheromone = 0

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0: # Check for valid edge
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta # Inverse of cost
                total_heuristic_pheromone += pheromone * heuristic

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0:
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta
                probability = (pheromone * heuristic) / total_heuristic_pheromone
                probabilities[neighbor] = probability
        return probabilities

    def _update_pheromones(self, all_paths):
        for path, cost in all_paths:
            if cost != float('inf'):
                pheromone_deposit = self.Q / cost
                for i in range(len(path) - 1):
                    self.pheromones[path[i]][path[i+1]] += pheromone_deposit

    def _evaporate_pheromones(self):
        self.pheromones *= (1 - self.evaporation_rate)

# Example Usage:
# Define a sample traffic graph (adjacency matrix with costs)
# 0: A, 1: B, 2: C, 3: D, 4: E
graph = np.array([
    [0, 10, 5, 0, 0],
    [0, 0, 0, 15, 0],
    [0, 3, 0, 0, 20],
    [0, 0, 0, 0, 8],
    [0, 0, 0, 0, 0]
])

aco = AntColonyOptimization(
    num_ants=10,
    num_iterations=100,
    alpha=1,
    beta=2,
    evaporation_rate=0.1,
    Q=100,
    graph=graph
)

start_node = 0 # Node A
end_node = 4   # Node E

best_path, best_cost = aco.run(start_node, end_node)
print(f"Best path found: {best_path}")
print(f"Cost of best path: {best_cost}")

Best path found: [0, 2, 1, 3, 4]
Cost of best path: 31


In [17]:
import numpy as np
import random
import matplotlib.pyplot as plt

class AntColonyOptimization:
    def __init__(self, num_ants, num_iterations, alpha, beta, evaporation_rate, Q, graph):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha  # Pheromone influence
        self.beta = beta    # Heuristic influence (inverse of travel time/distance)
        self.evaporation_rate = evaporation_rate
        self.Q = Q          # Pheromone deposit constant
        self.graph = graph  # Adjacency matrix of the traffic network
        self.num_nodes = len(graph)
        self.pheromones = np.ones((self.num_nodes, self.num_nodes))  # Initialize pheromones to 1

    def run(self, start_node, end_node):
        best_path = None
        best_path_cost = float('inf')

        for iteration in range(self.num_iterations):
            all_paths = []
            for ant in range(self.num_ants):
                path, cost = self._construct_path(start_node, end_node)
                all_paths.append((path, cost))

                if cost < best_path_cost:
                    best_path_cost = cost
                    best_path = path

            self._update_pheromones(all_paths)
            self._evaporate_pheromones()

            # Print the iteration details
            print(f"Iteration {iteration + 1}/{self.num_iterations}: Best path = {best_path}, Best cost = {best_path_cost}")

        return best_path, best_path_cost

    def _construct_path(self, start, end):
        current_node = start
        path = [current_node]
        visited = {current_node}
        path_cost = 0

        while current_node != end:
            probabilities = self._calculate_probabilities(current_node, visited)
            if not probabilities:  # No valid path
                return [], float('inf')

            next_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()), k=1)[0]
            path.append(next_node)
            path_cost += self.graph[current_node][next_node]
            visited.add(next_node)
            current_node = next_node
        return path, path_cost

    def _calculate_probabilities(self, current_node, visited):
        probabilities = {}
        total_heuristic_pheromone = 0

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0:  # Valid road
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta  # Inverse of cost (time)
                total_heuristic_pheromone += pheromone * heuristic

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0:
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta
                probability = (pheromone * heuristic) / total_heuristic_pheromone
                probabilities[neighbor] = probability
        return probabilities

    def _update_pheromones(self, all_paths):
        for path, cost in all_paths:
            if cost != float('inf'):
                pheromone_deposit = self.Q / cost
                for i in range(len(path) - 1):
                    self.pheromones[path[i]][path[i+1]] += pheromone_deposit

    def _evaporate_pheromones(self):
        self.pheromones *= (1 - self.evaporation_rate)

# Graph: A more complex traffic network (nodes: A=0, J=9)
graph = np.array([
    [0,  12, 8,  15, 0,  7,  5,  5,  2,  0],  # A
    [12, 0,  6,  0,  5, 0,  0,  7,  5,  0],  # B
    [8,  6,  0,  10, 6,  0,  0,  0,  0,  0],  # C
    [15, 5,  10, 0,  0,  14, 9,  0,  0,  0],  # D
    [0,  10, 6,  0,  0,  8,  5,  0,  0,  0],  # E
    [0,  0,  0,  14, 8,  0,  4,  0,  6,  0],  # F
    [0,  7,  0,  9,  5,  4,  0,  8,  0,  12], # G
    [7,  0,  0,  0,  0,  0,  8,  0,  9,  5],  # H
    [0,  0,  0,  0,  0,  6,  0,  9,  0,  11], # I
    [0,  0,  0,  0,  0,  0,  12, 5,  11, 0]   # J
])

# Create the ACO instance
aco = AntColonyOptimization(
    num_ants=20,
    num_iterations=50,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.1,
    Q=100,
    graph=graph
)

start_node = 0  # Node A
end_node = 9    # Node J

# Run the ACO algorithm
best_path, best_cost = aco.run(start_node, end_node)

print(f"Best path found: {best_path}")
print(f"Cost of best path: {best_cost}")


Iteration 1/50: Best path = [0, 7, 9], Best cost = 10
Iteration 2/50: Best path = [0, 7, 9], Best cost = 10
Iteration 3/50: Best path = [0, 7, 9], Best cost = 10
Iteration 4/50: Best path = [0, 7, 9], Best cost = 10
Iteration 5/50: Best path = [0, 7, 9], Best cost = 10
Iteration 6/50: Best path = [0, 7, 9], Best cost = 10
Iteration 7/50: Best path = [0, 7, 9], Best cost = 10
Iteration 8/50: Best path = [0, 7, 9], Best cost = 10
Iteration 9/50: Best path = [0, 7, 9], Best cost = 10
Iteration 10/50: Best path = [0, 7, 9], Best cost = 10
Iteration 11/50: Best path = [0, 7, 9], Best cost = 10
Iteration 12/50: Best path = [0, 7, 9], Best cost = 10
Iteration 13/50: Best path = [0, 7, 9], Best cost = 10
Iteration 14/50: Best path = [0, 7, 9], Best cost = 10
Iteration 15/50: Best path = [0, 7, 9], Best cost = 10
Iteration 16/50: Best path = [0, 7, 9], Best cost = 10
Iteration 17/50: Best path = [0, 7, 9], Best cost = 10
Iteration 18/50: Best path = [0, 7, 9], Best cost = 10
Iteration 19/50: Be

In [19]:
import numpy as np
import random
import matplotlib.pyplot as plt

class AntColonyOptimization:
    def __init__(self, num_ants, num_iterations, alpha, beta, evaporation_rate, Q, graph):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha  # Pheromone influence
        self.beta = beta    # Heuristic influence (inverse of travel time/distance)
        self.evaporation_rate = evaporation_rate
        self.Q = Q          # Pheromone deposit constant
        self.graph = graph  # Adjacency matrix of the traffic network
        self.num_nodes = len(graph)
        self.pheromones = np.ones((self.num_nodes, self.num_nodes)) * 0.1  # Initialize pheromones to small values (lower influence)

    def run(self, start_node, end_node):
        best_path = None
        best_path_cost = float('inf')

        for iteration in range(self.num_iterations):
            all_paths = []
            for ant in range(self.num_ants):
                path, cost = self._construct_path(start_node, end_node)
                all_paths.append((path, cost))

                if cost < best_path_cost:
                    best_path_cost = cost
                    best_path = path

            self._update_pheromones(all_paths)
            self._evaporate_pheromones()

            # Print the iteration details to observe the evolution
            print(f"Iteration {iteration + 1}/{self.num_iterations}: Best path = {best_path}, Best cost = {best_path_cost}")

        return best_path, best_path_cost

    def _construct_path(self, start, end):
        current_node = start
        path = [current_node]
        visited = {current_node}
        path_cost = 0

        while current_node != end:
            probabilities = self._calculate_probabilities(current_node, visited)
            if not probabilities:  # No valid path
                return [], float('inf')

            next_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()), k=1)[0]
            path.append(next_node)
            path_cost += self.graph[current_node][next_node]
            visited.add(next_node)
            current_node = next_node
        return path, path_cost

    def _calculate_probabilities(self, current_node, visited):
        probabilities = {}
        total_heuristic_pheromone = 0

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0:  # Valid road
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta  # Inverse of cost (time)
                total_heuristic_pheromone += pheromone * heuristic

        for neighbor in range(self.num_nodes):
            if neighbor not in visited and self.graph[current_node][neighbor] > 0:
                pheromone = self.pheromones[current_node][neighbor] ** self.alpha
                heuristic = (1.0 / self.graph[current_node][neighbor]) ** self.beta
                probability = (pheromone * heuristic) / total_heuristic_pheromone
                probabilities[neighbor] = probability
        return probabilities

    def _update_pheromones(self, all_paths):
        for path, cost in all_paths:
            if cost != float('inf'):
                pheromone_deposit = self.Q / cost
                for i in range(len(path) - 1):
                    self.pheromones[path[i]][path[i+1]] += pheromone_deposit

    def _evaporate_pheromones(self):
        self.pheromones *= (1 - self.evaporation_rate)

# Graph: A more complex traffic network with multiple competing paths (nodes: A=0, J=9)
graph = np.array([
    [0,  10, 10,  0,  0,  0,  0,  5,  0,  0],  # A
    [10, 0,  5,  0,  10, 0,  0,  0,  0,  0],  # B
    [10, 5,  0,  10, 6,  0,  0,  0,  0,  0],  # C
    [0,  0,  10, 0,  0,  10, 9,  0,  0,  0],  # D
    [0,  10, 6,  0,  0,  10, 5,  0,  0,  0],  # E
    [0,  0,  0,  10, 10, 0,  4,  0,  6,  0],  # F
    [0,  0,  0,  9,  5,  4,  0,  8,  0,  12], # G
    [5,  0,  0,  0,  0,  0,  8,  0,  9,  5],  # H
    [0,  0,  0,  0,  0,  6,  0,  9,  0,  11], # I
    [0,  0,  0,  0,  0,  0,  12, 5,  11, 0]   # J
])

# Create the ACO instance
aco = AntColonyOptimization(
    num_ants=20,
    num_iterations=50,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.1,  # Moderate evaporation rate
    Q=100,
    graph=graph
)

start_node = 0  # Node A
end_node = 9    # Node J

# Run the ACO algorithm
best_path, best_cost = aco.run(start_node, end_node)

print(f"Best path found: {best_path}")
print(f"Cost of best path: {best_cost}")


Iteration 1/50: Best path = [0, 7, 9], Best cost = 10
Iteration 2/50: Best path = [0, 7, 9], Best cost = 10
Iteration 3/50: Best path = [0, 7, 9], Best cost = 10
Iteration 4/50: Best path = [0, 7, 9], Best cost = 10
Iteration 5/50: Best path = [0, 7, 9], Best cost = 10
Iteration 6/50: Best path = [0, 7, 9], Best cost = 10
Iteration 7/50: Best path = [0, 7, 9], Best cost = 10
Iteration 8/50: Best path = [0, 7, 9], Best cost = 10
Iteration 9/50: Best path = [0, 7, 9], Best cost = 10
Iteration 10/50: Best path = [0, 7, 9], Best cost = 10
Iteration 11/50: Best path = [0, 7, 9], Best cost = 10
Iteration 12/50: Best path = [0, 7, 9], Best cost = 10
Iteration 13/50: Best path = [0, 7, 9], Best cost = 10
Iteration 14/50: Best path = [0, 7, 9], Best cost = 10
Iteration 15/50: Best path = [0, 7, 9], Best cost = 10
Iteration 16/50: Best path = [0, 7, 9], Best cost = 10
Iteration 17/50: Best path = [0, 7, 9], Best cost = 10
Iteration 18/50: Best path = [0, 7, 9], Best cost = 10
Iteration 19/50: Be